# Tax Calculator
### Following documentation found here: https://github.com/justinmart/simple_taxes

This project simplifies the process of determining profit-and-loss for cryptocurrency trades. It will provide:

A detailed report containing information about every trade
A detailed profit-and-loss calculation on every trade
A simplified report containing basic profit-and-loss data on each trade (useful for Tax Accountants or for tax filings)
A report on aggreagate profit-and-loss from all trades by year, and by short-term vs long-term gains
A report on all errors encountered in the process

It supports trades from all the major US Exchanges and some international exchanges via csv and xlsx trading history files.

Coinbase, Coinbase Pro, Binance, Gemini, Kraken, Poloniex, Bitfinex, Bittrex, Circle, and Liqui. It also supports manual trades via entering them into a spreadsheet and saving a copy as a .csv

These instructions will get you a copy of the project up and running on your local machine, and guidance on how to obtain your own tax reports.

* Go to each supported exchange for which you have trades and download a csv or xlsx of your trade history
* Put these csv or xlsx files into the appropriate folder inside the data/ directory (the program will automatically parse all files in the correct folders)
* For any trades that are from unsupported exchanges, or any other manual trades (ICOs, forks, in-person or on-chain trades, etc) populate a spreadsheet with a specific format (see full documentation as provided in the link above)
* Run the script in this jupyter notebook, which populates 5 files in the output folder:
    - trades.csv : Detailed information regarding every trade across all exchanges and currencies
    - errors : Information on any errors or any trades left out of the calculations
    - pnl.cvs : Profit-and-loss calculations for each trade, including long-term vs short-term gains
    - total_pnl : Aggregate information on profit and loss for each year, by short term vs long term
    - tax_reporting_data : A spreadsheet intended to be sent to the IRS or a tax accountant which details every trade and the profit-and-loss associated with each

In [ ]:
from taxes import Taxes

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# Set the current year
current_year = 2018

In [ ]:
trades, errors, pnl, total_pnl, tax_reporting_data, remaining_funds = Taxes().run()

In [ ]:
print "Latest trades recorded:\n"
for _ in trades.groupby(['platform'])['created_at'].max().iteritems():
    print "{:<40}".format(_[0]), _[1]

In [ ]:
total_pnl

In [ ]:
print "Total PNL: ", int(total_pnl.loc[current_year].sum())

In [ ]:
print "Current PNL contributions for"
a = pnl[pnl['year'] == current_year].copy()
a.groupby(['currency']).pnl.sum().sort_values(ascending=False)

# Sales Calculator
## Calculates how selling crypto assets will affect your capital gains. 
### (1) How much can you sell to minimize total capital gains for this year?
### (2) How much can you sell to maximize total profit while not impacting your current capital gains amount? 

#### In order to use the Sales Calculator, you need to obtain a free API key from Coinmarketcap.com
Go go https://coinmarketcap.com/api/ and sign up for a their free basic API. Enter the provided key on the current_exchange_rates.py file on line 6. 

In [ ]:
import current_exchange_rates
import sales_calculator
reload(sales_calculator)


# pnl_threshold:     With LIFO, when selling crypto assets the first few sales could increase 
#                    PNL before getting to other assets that would decrease PNL. This threshold 
#                    allows you increase PNL up to <X> before stopping the process. 
#                    Default is $10,000
threshold = 10000

In [ ]:
sales_auth = sales_calculator.SalesCalculator(year=current_year)

In [ ]:
sales = sales_auth.sales_data(
    pnl=pnl,
    remaining_funds=remaining_funds,
    pnl_threshold=threshold,
)

In [ ]:
sales_summary = sales_auth.find_min_max_pnl(sales)

In [ ]:
for _ in sales.currency.unique():
    print "Sales Simulator for {}".format(_)
    temp = sales[sales['currency'] == _].copy()
    plt.plot(temp['rolling_amount'], temp[['rolling_sales','rolling_pnl']])
    plt.legend(['Total $ Sold', 'Total PNL Impact']) 
    plt.show()

In [ ]:
sales_auth.final_pnl_and_sales(
    coins_to_sell=[_ for _ in sales_summary.keys() if _ not in ['total_min', 'total_max']],
    summary=sales_summary,
    total_pnl=pnl
)

In [ ]:
print "Current holdings (based on your trading history):"
for _ in remaining_funds.currency.unique():
    print "{:<5}".format(_), " {:<20}".format(remaining_funds[remaining_funds['currency'] == _].amount.sum())